In [1]:
%%init_spark
launcher.jars = ["libs/kensu-dam-spark-collector-0.17.2_spark-3.0.1.jar"]


In [2]:
val properties = {
    import scala.collection.JavaConverters._
    val p = new java.util.Properties()
    p.load(new java.io.FileInputStream("./application.properties"))
    val m = scala.collection.mutable.Map.empty[String, Any]
    m++=p.asScala.mapValues(v => scala.util.Try(v.toBoolean).orElse(scala.util.Try(v.toInt)).getOrElse(v))
    m
}
val dam_url = 
val token = ""
val processName = "taratata"
val mocked_timestamp = System.currentTimeMillis

properties.put("dam.ingestion.auth.token", token)
properties.put("dam.activity.code.maintainers", List(sys.env("USER")))
properties.put("dam.activity.code.version", System.currentTimeMillis)
properties.put("dam.activity.user", sys.env("USER"))
properties.put("dam.activity.organization", "Unknown")
properties.put("dam.activity.projects", properties("dam.activity.projects").toString.split(","))
properties.put("dam.activity.explicit.process.name", processName)
properties.put("dam.activity.explicit.process_run.name", processName)

io.kensu.third.integration.TimeUtils.setMockedTime(mocked_timestamp, false)

val w = io.kensu.dam.lineage.spark.lineage.Implicits.SparkSessionDAMWrapper(spark)
w.track(Some(dam_url), None)(properties.toList:_*)

Intitializing Scala interpreter ...

Spark Web UI available at http://jupyter-kensuio-2doss-2ddodd-2dworkshop-2d6frk2s67:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1636124412003)
SparkSession available as 'spark'


maybeDamFileDebugLevel: Some(DEBUG)
Activating DAM debug logging to file: debug.log
Done activating DAM debug logging to a file
Creating a new DAM scala client actor-system
Done creating a new DAM scala client actor-system.


properties: scala.collection.mutable.Map[String,Any] = Map(dam.activity.spark.environnement.provider -> io.kensu.dam.lineage.spark.lineage.DefaultSparkEnvironnementProvider, dam.activity.code.repository -> binder://workshop-dodd, dam.activity.organization -> Unknown, dam.activity.explicit.process.name -> taratata, dam.spark.data_stats.input.only_used_in_lineage -> true, dam.spark.file_debug.file_name -> debug.log, dam.datasources.path_rules.short.naming_strategy -> "", dam.activity.explicit.process_run.name -> taratata, dam.spark.data_stats.enabled -> true, dam.spark.data_stats.output.coalesceEnabled -> false, dam.ingestion.is_offline -> true, dam.ingestion.entity.compaction -> true, dam.activity.user -> jovyan, dam.ingestion.auth.token -> eyJhbGciOiJIUzI1NiJ9.eyIkaW50X3Blcm1zIjpbXSwic3...


In [3]:
val week = "week1"

week: String = week1


In [4]:
val df_customer = spark.read.option("inferSchema","true").option("header","true").csv(s"./data/${week}/custinfo.csv")

df_customer: org.apache.spark.sql.DataFrame = [id: int, first_name: string ... 3 more fields]


In [5]:
val df_transaction = spark.read.option("inferSchema","true").option("header","true").csv(s"./data/${week}/transactions.csv")

df_transaction: org.apache.spark.sql.DataFrame = [id: int, date: string ... 3 more fields]


In [6]:
val df_join = df_customer.join(df_transaction, df_customer("id") === df_transaction("id"),"inner").drop(df_transaction("id"))

df_join: org.apache.spark.sql.DataFrame = [id: int, first_name: string ... 7 more fields]


In [7]:
df_join.count()

res1: Long = 1000


In [8]:
df_join.write.mode("overwrite").save(s"./data/${week}/joined_data")

In [9]:
val df = spark.read.option("inferSchema","true").option("header","true").load(s"./data/${week}/joined_data")

df: org.apache.spark.sql.DataFrame = [id: int, first_name: string ... 7 more fields]


In [10]:
df.show()

+---+----------+-----------+--------------------+-----------+----------+--------------------+-----+--------+
| id|first_name|  last_name|               email|     gender|      date|             product|price|quantity|
+---+----------+-----------+--------------------+-----------+----------+--------------------+-----+--------+
|  1|    Halsey|   Andreone|handreone0@printf...|    Agender|10/24/2021|Lettuce - Boston ...|  8.8|       1|
|  2|       Web|  Fountaine|wfountaine1@twitp...|    Agender|10/21/2021|Wine - Cave Sprin...| 10.2|       3|
|  3|      Pepe|   Skoggins|pskoggins2@diigo.com|     Female|10/18/2021|Cookie Dough - Pe...| 10.9|       1|
|  4|    Goldie|     Chevis|gchevis3@hatena.n...|Genderfluid|10/19/2021|Triple Sec - Mcgu...| 14.2|       5|
|  5|      Joli|Matuszewski|jmatuszewski4@4sh...| Non-binary|10/23/2021|Beer - Sleemans H...| 16.8|       5|
|  6|    Crysta|     Wybern|cwybern5@eventbri...|     Female|10/18/2021|Syrup - Kahlua Ch...|  8.1|       5|
|  7|      Evyn|   

In [11]:
df.createOrReplaceTempView("df")

In [12]:
val Total_DF = spark.sql("SELECT id,price,quantity,round(price * quantity,2) AS total FROM df")

Total_DF: org.apache.spark.sql.DataFrame = [id: int, price: double ... 2 more fields]


In [13]:
Total_DF.show()

+---+-----+--------+-----+
| id|price|quantity|total|
+---+-----+--------+-----+
|  1|  8.8|       1|  8.8|
|  2| 10.2|       3| 30.6|
|  3| 10.9|       1| 10.9|
|  4| 14.2|       5| 71.0|
|  5| 16.8|       5| 84.0|
|  6|  8.1|       5| 40.5|
|  7| 18.0|       3| 54.0|
|  8|  5.5|       2| 11.0|
|  9| 15.9|       2| 31.8|
| 10|  7.9|       3| 23.7|
| 11| 12.2|       4| 48.8|
| 12|  1.6|       4|  6.4|
| 13|  1.0|       1|  1.0|
| 14|  2.6|       2|  5.2|
| 15| 16.1|       2| 32.2|
| 16| 14.0|       5| 70.0|
| 17| 13.0|       3| 39.0|
| 18| 10.4|       5| 52.0|
| 19| 13.1|       1| 13.1|
| 20| 14.4|       3| 43.2|
+---+-----+--------+-----+
only showing top 20 rows



In [14]:
Total_DF.write.mode("overwrite").save(s"./data/${week}/data_total")